<a href="https://colab.research.google.com/github/mefiuuu1/OOPLabs2024/blob/main/proiect_bac.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
from datasets import Dataset
import os

def incarca_date():
    dataset = []
    # Textul integral
    with open("/content/drive/MyDrive/proiect_bac/ion/text_integral.txt", "r", encoding="utf-8") as f:
        dataset.append(f.read())
    # Comentarii
    for root, dirs, files in os.walk("/content/drive/MyDrive/proiect_bac/ion/comentarii"):
        for file in files:
            with open(os.path.join(root, file), "r", encoding="utf-8") as f:
                dataset.append(f.read())
    print(f"Am încărcat {len(dataset)} texte.")  # Verifică câte texte au fost încărcate
    return dataset

# 1. Setează pad_token
tokenizer = GPT2Tokenizer.from_pretrained("readerbench/RoGPT2-medium")
tokenizer.pad_token = tokenizer.eos_token

# 2. Încarcă modelul
model = GPT2LMHeadModel.from_pretrained("readerbench/RoGPT2-medium")

# 3. Tokenizare cu padding și truncation
texts = incarca_date()
inputs = tokenizer(
    texts,
    return_tensors="pt",
    padding="max_length",
    truncation=True,
    max_length=512
)
print(inputs)  # Verifică ce returnează tokenizer-ul

# 4. Antrenare

train_dataset = Dataset.from_dict({
    "input_ids": inputs["input_ids"],
    "attention_mask": inputs["attention_mask"],
    "labels": inputs["input_ids"],  # For language modeling, labels are the same as input_ids
})

training_args = TrainingArguments(
    output_dir="model-ion",
    per_device_train_batch_size=2,
    num_train_epochs=500,
    learning_rate=3e-5,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)
trainer.train()

# Save the tokenizer and model configuration after training
tokenizer.save_pretrained("model-ion")  # Save tokenizer to "model-ion" directory
model.save_pretrained("model-ion")  # Save model to "model-ion" directory

Am încărcat 7 texte.
{'input_ids': tensor([[26661,   352,  1149,  ...,   607,  4816,    12],
        [ 5097,  8828, 41708,  ...,     0,     0,     0],
        [ 5097,  8828, 31467,  ...,     0,     0,     0],
        ...,
        [   48, 21916, 45114,  ...,     0,     0,     0],
        [   48, 21916, 45114,  ...,     0,     0,     0],
        [   48, 21916, 45114,  ...,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}


Step,Training Loss
500,0.205400
1000,0.003200
1500,0.002200
2000,0.001900


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [66]:
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, AutoConfig

# Încarcă tokenizer-ul mai întâi
tokenizer = AutoTokenizer.from_pretrained("model-ion", trust_remote_code=True)  # Add trust_remote_code=True

# Încarcă configurația modelului
config = AutoConfig.from_pretrained("model-ion", trust_remote_code=True)

# Apoi, încarcă modelul folosind configurația
model = AutoModelForCausalLM.from_pretrained("model-ion", trust_remote_code=True, config=config)  # Add trust_remote_code=True

# Salvare explicită a configurației și tokenizer-ului în timpul antrenării
# (Adaugă aceste linii în scriptul de antrenare după trainer.train())
tokenizer.save_pretrained("model-ion")
model.config.save_pretrained("model-ion")


# Creează generatorul de text
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Generează un comentariu
prompt = """
Redactează un eseu de minimum 400 de cuvinte, în care să prezinți particularități ale romanului „Ion” de Liviu Rebreanu.

În elaborarea eseului, vei avea în vedere următoarele repere:

– Evidențierea a două trăsături distinctive care fac posibilă încadrarea romanului „Ion” într-o anumită perioadă istorică, într-un curent literar sau într-o orientare tematică specifică (de exemplu, realismul, contextul rural sau lupta pentru pământ).
– Comentarea a două episoade sau secvențe relevante din roman, care evidențiază temele principale și complexitatea personajelor, cum ar fi conflictul dintre destin și voința individuală sau relațiile tensionate dintre personaje.
– Analiza a două elemente de structură, compoziție și/sau de limbaj, semnificative pentru romanul „Ion”. Acestea pot include, de exemplu, analiza modului în care acțiunea este organizată, conflictul central, modul de redare a relațiilor temporale și spațiale, tehnicile narative folosite (cum ar fi perspectiva narativă sau registrele stilistice) și impactul acestora asupra înțelegerii mesajului autorului.

Răspuns:
"""
comentariu = generator(
    prompt,
    max_length=700,
    num_return_sequences=1,
    temperature=0.7,
    top_k=100,
    repetition_penalty=1.2)[0]["generated_text"]
print(comentariu)

Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.



Redactează un eseu de minimum 400 de cuvinte, în care să prezinți particularități ale romanului „Ion” de Liviu Rebreanu.

În elaborarea eseului, vei avea în vedere următoarele repere:

– Evidențierea a două trăsături distinctive care fac posibilă încadrarea romanului „Ion” într-o anumită perioadă istorică, într-un curent literar sau într-o orientare tematică specifică (de exemplu, realismul, contextul rural sau lupta pentru pământ).
– Comentarea a două episoade sau secvențe relevante din roman, care evidențiază temele principale și complexitatea personajelor, cum ar fi conflictul dintre destin și voința individuală sau relațiile tensionate dintre personaje.
– Analiza a două elemente de structură, compoziție și/sau de limbaj, semnificative pentru romanul „Ion”. Acestea pot include, de exemplu, analiza modului în care acțiunea este organizată, conflictul central, modul de redare a relațiilor temporale și spațiale, tehnicile narative folosite (cum ar fi perspectiva narativă sau registrel